## Кластеризация фильмов

In [137]:
import pandas as pd
import numpy as np

import time

from matplotlib import cm

import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls

import sklearn.manifold as sm
import sklearn.decomposition as sd
import sklearn.cluster as sc

Считываем данные. Изначально дата сет взят отсюда http://grouplens.org/datasets/movielens/ (1М), но я немного преобразовал данные, так, чтобы каждая строка соответствовала одному фильму. 

In [138]:
user_cols = ["U%d" % i for i in xrange(6040)]
movies_df = pd.read_csv("ml/movies.data", header=None, sep=",", names=["title", "genres"] + user_cols)
movies_df.head()

,title,genres,U0,U1,U2,U3,U4,U5,U6,U7,...,U6030,U6031,U6032,U6033,U6034,U6035,U6036,U6037,U6038,U6039
0,Toy Story (1995),Animation|Children's|Comedy,5,0,0,0,0,4,0,4,...,0,4,0,0,4,0,0,0,0,3
1,Jumanji (1995),Adventure|Children's|Fantasy,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,Waiting to Exhale (1995),Comedy|Drama,0,0,0,0,0,0,0,3,...,0,0,0,0,2,2,0,0,0,0
4,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


### Кластеризация данных
Для начала кластеризуем фильмы с помощью mini-batch k-means. Применение `partial_fit` сделано намеренно, чтобы подчеркнуть возможность применения этого алгоритма онлайн (хотя класс из коробки поддерживает мини-батчи). Также померяем время работы кластеризатора.

In [139]:
X = movies_df[user_cols].values.astype(float)

def cluster_k_means(x, k, batch_size=100):    
    km = sc.MiniBatchKMeans(n_clusters=k)
    for j in xrange(X.shape[0]/batch_size):
        print "Iteration %d" % j
        x_b = x[j * batch_size: max((j+1) * batch_size, x.shape[0])]
        km.partial_fit(x_b)
    return km.predict(x)    

start = time.time()
Y1 = cluster_k_means(X, 18)
print "Clustering took %s sec." % (time.time() - start)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Clustering took 10.5890388489 sec.


Примените алгоритм [BIRCH](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.Birch.html) к тем же данным. Быстрее ли он работет?

In [140]:
# TODO: Apply BIRCH algorighm here. Y2 - результат кластеризации
Y2 = np.zeros(X.shape[0], dtype=int)

### Визуализация результатов кластеризации
Для визуализации результатов применим алгоритм t-SNE. Предварительно данные нужно подготовить с помощью PCA (2 семестр). В итоге мы получим двумерные векторы.

In [141]:
pca = sd.PCA(n_components=500)
Z = pca.fit_transform(X)
tsne = sm.TSNE(n_components=2, n_iter=10000, perplexity=5, early_exaggeration=10, learning_rate=250, verbose=1)
W = tsne.fit_transform(Z)

[t-SNE] Computing pairwise distances...
[t-SNE] Computed conditional probabilities for sample 1000 / 2915
[t-SNE] Computed conditional probabilities for sample 2000 / 2915
[t-SNE] Computed conditional probabilities for sample 2915 / 2915
[t-SNE] Mean sigma: 3.458680
[t-SNE] Error after 100 iterations with early exaggeration: 78.605018
[t-SNE] Error after 349 iterations: 3.458238


Далее нарисуем результат кластеризации с помощью библиотеки [plotly](https://plot.ly/). Прежде чем начать работу необходимо установить эту библиотеку и сделать шаги из этого [туториала](https://plot.ly/python/getting-started/).

При наведении на точку на графике будет появляться название фильма и список жанров. Чтобы нарисовать это, нам понадобится список из строк, содержащих необходимую информацию, в том же самом порядке, в каком идут фильмы в наших данных.

In [142]:
names = [
    movies_df["title"][i].decode('utf-8', errors='ignore') 
    + "<br>" 
    + movies_df["genres"][i] for i in xrange(movies_df.shape[0])
]

Аналогично каждой точке будет соответствовать цвет, определяемый по номеру кластера. В текущей реализации этот цвет один для всех фильмов. Вам предлагается реализовать распределение цветов, использовав для этого результаты кластеризации `Y1` или `Y2`.

In [143]:
def get_color(i, N):
    r, g, b, a = cm.Set1(256 / (N - 1) * i)
    return "rgb(%d, %d, %d)" % (int(256*r), int(256*g), int(256*b))

colors = [get_color(1, 2) for n in names]

Наконец размер точки будет больше для популярных фильмов. Блок кода ниже реализует это.

In [144]:
sizes = (2 * np.log((X > 0).sum(axis=0))).tolist()

Строим картинку. 
1) Появляются ли близкие по жанру фильмы по соседству друг с другом?
2) Появляются ли точки одного цвета по соседству друг с другом?
3) Как отличаются кластеризации с помощью k-means и birch?
Замечание: ploly позволяет увеличивать области на графике.

In [145]:
trace0 = Scatter(
    x=W[:, 0],
    y=W[:, 1],
    text=names,
    mode='markers',
    marker=Marker(
        color=colors,
        size=sizes
    )
)
data = Data([trace0])
layout = Layout(
    showlegend=False,
    height=1200,
    width=600,
)
fig = Figure(data=data, layout=layout)
plot_url = py.plot(fig, filename='bubblechart-text')

tls.embed("https://plot.ly/~anokhinn/173", height=1000)